In [ ]:
import numpy as np

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib.common.recurrent.type_aliases import RNNStates
import gym
import torch as th

In [ ]:
from active_critic.utils.gym_utils import make_policy_dict

In [ ]:
env = gym.make("CartPole-v1")

In [ ]:
model = RecurrentPPO("MlpLstmPolicy", "CartPole-v1", verbose=1)

In [ ]:
lstm_stats = model._last_lstm_states

In [ ]:
obs =  th.tensor(env.reset(),device='cuda').reshape([1,-1])

In [ ]:
a = model.policy.train()

In [ ]:
a = model.policy.train()
for j in range(1000):
    rnn_states_list = [RNNStates(pi=lstm_stats[0], vf=lstm_stats[1])]

    for i in range(10):
        actions, values, log_prob, rnn_states = model.policy.forward(obs = obs, lstm_states=rnn_states_list[-1], episode_starts = th.zeros([1], device='cuda'), deterministic=False)
        rnn_states_list.append(rnn_states)

    model.policy.optimizer.zero_grad()
    total_loss = 0
    for i in range(len(rnn_states_list) - 1):
        if i == 0:
            episode_starts = th.ones([1], device='cuda')
        else:
            episode_starts = th.zeros([1], device='cuda')

        if i%2 == 0:
            wanted_action = th.ones_like(actions)
        else:
            wanted_action = th.zeros_like(actions)

        value, log_prob, entropy = model.policy.evaluate_actions(obs, wanted_action, rnn_states_list[i], episode_starts = episode_starts)
        prob_true_act = th.exp(log_prob).mean()
        log_prob = log_prob.mean()
        entropy = entropy.mean()


        ent_loss = 1e-2 * entropy
        neglogp = -log_prob
        loss = neglogp + ent_loss
        total_loss = total_loss + loss
    total_loss = total_loss.mean()
    if j%100 == 0:
        print(f'total_loss {total_loss}')
    total_loss.backward()
    model.policy.optimizer.step()


In [27]:
o, states = model.policy.predict(obs.cpu().numpy())

In [28]:
o

array([1])

In [29]:
o, states = model.policy.predict(obs.cpu().numpy(), state=states)

In [30]:
o

array([0])

In [31]:
o, states = model.policy.predict(obs.cpu().numpy(), state=states)

In [32]:
o

array([1])

In [ ]:
loss.mean().backward()

In [ ]:
rnn_states_list[-1][0][0].mean().backward()

In [ ]:
model.policy.lstm_actor.all_weights[0][0]._grad

In [ ]:
ent_weight = 1e-2

eval actions:         
obs: th.Tensor,
actions: th.Tensor,
lstm_states: RNNStates,
episode_starts: th.Tensor,

-> 
values, log_prob, distribution.entropy()

In [ ]:
_, log_prob, entropy = model.policy.evaluate_actions(obs, actions, rnn_states, episode_starts = th.ones([1], device='cuda'))
#print(f'max log: {log_prob.max()}')

prob_true_act = th.exp(log_prob).mean()
log_prob = log_prob.mean()
entropy = entropy.mean()


ent_loss = ent_weight * entropy
neglogp = -log_prob
loss = neglogp + ent_loss

In [ ]:
loss

In [ ]:
model.policy.lstm_actor.forward(obs = env.reset(), lstm_states=lstm_stats, epsidode_starts = th.zeros([1]))

In [ ]:
from imitation.algorithms.bc import BC
from active_critic.utils.gym_utils import (
    make_vec_env,
    make_dummy_vec_env,
    sample_expert_transitions_rollouts,
    make_pomdp_rollouts,
    make_dummy_vec_env_pomdp,
    get_avr_succ_rew_det
)

In [ ]:
env_tag = 'pickaplce'
seq_len = 100
n_demonstrations

env, vec_expert = make_dummy_vec_env(name=env_tag, seq_len=seq_len)
val_env, _ = make_dummy_vec_env(name=env_tag, seq_len=seq_len)

transitions, rollouts = sample_expert_transitions_rollouts(
    vec_expert.predict, val_env, n_demonstrations)